<h1> Creating a Simple SQL Database, Table and Running SQL queries </h1>

In [5]:
#!pip install ipython-sql

#importing all the relevant libraries

import pandas as pd
import sqlite3
import datetime as dt
import numpy as np

In [6]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


<h2> Reading the skincare dataset </h2>

- The dataset obtained has been scraped from https://www.beautybay.com/ 
- It contains a product_type, brand_name, product_description, price and quantity columns
- There are five types of products (cleansers, moisturizers, sunscreen/uv, masks, toner/mist and tools
- There are 87 unique brands ('Carbon Theory', 'Mario Badescu', 'forence by is', 'Dr. Paw Paw',
       'By BEAUTY BAY', 'Yes To', 'aya skin', 'Faracy', 'Eeis',
       'Forua 10.0.6', 'Revoution Skincare', 'COSRX',
       'Jeffree Star Cosetics', 'NIOD', 'The Ordinary', 'Apha-H',
       'Hoika Hoika', 'InstaNatura', 'Antipodes', 'Anthony',
       'Aureia Probiotic Skincare', 'DERMAdoctor', 'Dr. Hauschka',
       'Discipe', 'Dr. Botanicas', 'Ebryoisse', 'Foreo', 'Freck', 'rüu',
       'HoiFro', 'Institut Estheder', 'L.A. Gir', 'Mehron',
       'Naturay Serious', 'Neihbourhood Botanicas', 'Ofra', 'Oh K!',
       'PSA Skin', 'Project Lip', 'Peter Thoas Roth', 'Revoution Pro',
       'Sand & Sky', 'STARSKIN', 'TONYMOLY', 'UpCirce Beauty',
       'West Barn Co', 'NUDESTIX', 'Physicians Forua', 'Cowshed', 'Oskia',
       'Taika', 'Generation Cay', 'Jouer Cosetics', 'Saturday Skin',
       'Skin79', 'Brushworks', 'Mount Lai', 'Skin Gy', 'Sia Beauty',
       'ECookin', 'Kitsch', 'BH Cosetics', 'Cover FX', 'Farsai',
       'Linda Haber', 'Miani', 'Revolution Skincare', 'Ultrasun',
       'Alpha-H', 'Holika Holika', 'Paul & Joe', 'Elemis',
       'Institut Esthederm', 'Embryolisse', 'Soleil Toujours',
       'Formula 10.0.6', 'Dr. Lipp', 'Eyptian Maic', 'Jaaican Mano & Lie',
       'Aies of Skin', 'Anastasia Bevery His', 'NEOM', 'Pau & Joe',
       'Makeup Revoution', 'TAN-LUXE', 'Utrasun', 'Soei Toujours')
       
       
- The product description column describes what a particular product is (examples include 'Get That Grie Face Scrub', Botanica Exfoliatin Scrub' and 'AHA 30% + BHA 2% Peeling Soution'
       
  


In [53]:
#reading csv file

skincare_df_new=pd.read_csv('../datasets/skincare_df_new.csv')

<h2> Creating a python function to clean the data</h2>

The function removes:
    
    - Extra columns
    -creates a new index
    - strips $ sign from the price column
    
    
    

In [54]:
#creating a python function to remove unnamed column, prod_idx and dollar sign in price

def clear(df):
    df.drop(columns=['Unnamed: 0',
                    'prod_idx'],inplace=True)
    df.reset_index(inplace=True)
    df.rename(columns={'index':'id'},inplace=True) #renaming index col
    
    df['price']=df['price'].apply(lambda x:x.strip('$'))
    
    return df.reset_index()



<h2> Applying function to the skincare dataset </h2>

In [ ]:
#applying the python function
clear(skincare_df_new)

<h2> Creation of Connection and Cursor </h2>

In [144]:
#defining connection and cursor

connection=sqlite3.connect('skincare_p.db',timeout=10)

cursor=connection.cursor()

<h2> Creation of skincare table </h2>

In [145]:
#creating a table for all skincare products

command_1="""CREATE TABLE IF NOT EXISTS 
skincare(id SERIAL PRIMARY KEY,
product_type TEXT NOT NULL,
brand_name VARCHAR(50) NOT NULL,
product_description VARCHAR(100) NOT NULL,
price DECIMAL NOT NULL,
quantity INTEGER NOT NULL
)

"""

cursor.execute(command_1)

In [152]:
cursor.execute(command_1)

In [147]:
connection.commit()

<h2> Table is empty since no data has been fed </h2>

In [148]:
#checking if table exists
cursor.execute("""
SELECT *
FROM skincare""")

results=cursor.fetchall()
print(results)  #since we have not added any data, it is empty

[]


<h2> Adding new entries into skincare table </h2>

In [151]:
add_new_entry="""
       INSERT INTO skincare
       (id,product_type,brand_name,product_description,price,quantity)
       VALUES(?,?,?,?,?,?)


"""

for index,row in skincare_df_new.iterrows():
    sub_values = (row['id'],row['product_type'],row['brand_name'],row['product_description'],
                  row['price'],row['quantity'])
    
    cursor.execute(add_new_entry,sub_values)
    
connection.commit()

<h3> Now the table is ready and we can run our queries </h3>

<h2> Query 1) </h2>

Reading a simple Query 

In [155]:
#reading a simple query

pd.read_sql_query("""SELECT *
                    FROM skincare
                    LIMIT 5""",connection)
                           

,id,product_type,brand_name,product_description,price,quantity
0,0,cleansers,Carbon Theory,Ceansin Bar,9.00,100
1,1,cleansers,Mario Badescu,Acne Facia Ceanser,20.25,177
2,2,cleansers,forence by is,Get That Grie Face Scrub,13.50,50
3,3,cleansers,Dr. Paw Paw,Scru Nourish,12.00,16
4,4,cleansers,Mario Badescu,Botanica Exfoiatin Scrub,34.50,100


<h2> Query 2)  </h2>

Finding Average prices by product Type

In [156]:
pd.read_sql_query(""" SELECT product_type, round(avg(price),2) as avg_price
                      FROM skincare
                      GROUP BY 1
                      ORDER BY 2 DESC
                      


                """,connection)

,product_type,avg_price
0,tools,120.83
1,sunscreen/uv,38.79
2,moisturizers,36.77
3,masks,28.77
4,cleansers,24.53
5,toner/mist,23.10


<b> Inference: </b>
    
From the above table, tools are considered the most expensive product to be sold on beautybay website. Toners/mist on the other hand are the most affordable products sold

<h2> Query 3) </h2>

Hypothetical Customer A is looking for an AHA peel for exfoliation and is looking for products below $30.
Execute a query to provide a few recommendations 

In [175]:
pd.read_sql_query(""" SELECT brand_name, price, product_description
                     FROM skincare
                     WHERE product_description LIKE '%AHA%'
                     and price<30
                     ORDER BY 2
                    
                    
                """,connection)

,brand_name,price,product_description
0,The Ordinary,10,AHA 30% + BHA 2% Peein Soution
1,The Ordinary,10,AHA 30% + BHA 2% Peein Soution
2,Mario Badescu,24,Roin Crea Pee With AHA
3,Mario Badescu,27,AHA & Ceraide Moisturizer


<b> Inference </b> 

Products from both The Ordinary and Mario Badescu could be recommended to the customer
looking for products with AHA 



<h2> Query 4) </h2>

Ranking the products by product type from most affordable to the most expensive product



In [273]:
pd.read_sql_query(""" SELECT brand_name,product_description, product_type,
                       DENSE_RANK() OVER(PARTITION BY product_type
                       ORDER BY price DESC 
                       ) as rank, price
                     FROM skincare
                """,connection)



,brand_name,product_description,product_type,rank,price
0,Peter Thoas Roth,Peptide 21 Aino Acid Exfoiatin Pee Pads,cleansers,1,67.75
1,Sand & Sky,Austraian Eu Appe Enzye Powder Poish,cleansers,2,67.25
2,Eeis,Pro-Coaen Ceansin Ba,cleansers,3,64.25
3,Eeis,Pro-Coaen Rose Ceansin Ba,cleansers,3,64.25
4,Aureia Probiotic Skincare,Mirace Ceanser,cleansers,4,63.00
...,...,...,...,...,...
1013,Sia Beauty,S20 Eye Crea Brush,tools,20,16.50
1014,Brushworks,HD Backhead & Beish Reover Set,tools,21,15.00
1015,Sia Beauty,S02 Spatua Brush,tools,22,13.00
1016,Kitsch,Eco-Friendy Derapaners,tools,23,10.75


<h2> Query 5) </h2>

Find minimum, avg and maximum product price by brand

In [191]:
pd.read_sql_query("""
                SELECT brand_name, round(min(price),1) as min,
                round(avg(price),1) as avg,round(max(price),1) as max
                FROM skincare
                GROUP BY brand_name
              """,connection)

,brand_name,min,avg,max
0,Aies of Skin,108.3,108.3,108.3
1,Alpha-H,24.8,30.2,34.5
2,Anastasia Bevery His,53.7,53.7,53.7
3,Anthony,12.0,43.9,82.8
4,Antipodes,10.3,30.3,52.3
...,...,...,...,...
82,West Barn Co,30.0,30.0,30.0
83,Yes To,2.5,11.7,18.0
84,aya skin,18.8,29.1,45.0
85,forence by is,9.0,20.9,33.0


<h2> Query 6 </h2>

Find the most affordable and expensive products and price by brand name

In [235]:
pd.read_sql_query("""
                SELECT brand_name, 
                min(price) as price,
                min(product_description) as affordable_product,
                
                max(price) as price_max,
                max(product_description) as most_expensive_product
        
                FROM skincare
                GROUP BY brand_name
              """,connection)

,brand_name,price,affordable_product,price_max,most_expensive_product
0,Aies of Skin,108.25,Muti Hyauronic Antioxidant Hydration Seru,108.25,Muti Hyauronic Antioxidant Hydration Seru
1,Alpha-H,24.75,Absolute Lip Perfector,34.50,Protection Plus Daily SPF 50+
2,Anastasia Bevery His,53.70,Hydratin Oi,53.70,Hydratin Oi
3,Anthony,12.00,A-Purpose Facia Moisturizer,82.75,Purifyin Astrinent Pads
4,Antipodes,10.25,Aura Manuka Honey Mask,52.25,Vania Pod Hydratin Day Crea
...,...,...,...,...,...
82,West Barn Co,30.00,Bo Myrte And Lie Baancin Ceanser,30.00,Bo Myrte And Lie Baancin Ceanser
83,Yes To,2.50,Coconut & Coffee 2-in-1 Scrub & Ceanser Stick,18.00,Yes To Coconut Oi Ceansin Ba
84,aya skin,18.75,Austraian Native Berries Moisturiser,45.00,Poeranate Facia Exfoiator
85,forence by is,9.00,16 Wishes Better Toether Pee Off Mask Duo,33.00,Zero Chi Face Mist


<h3> The End </h3>